In [1]:
import os
import numpy as np
import pandas as pd
from time import sleep
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from library.mapImaging import Get_Style_Map_Image_Between,get_poi_feat

In [2]:
style="maptype=roadmap&style=element:labels%7Cvisibility:off&style=feature:administrative.land_parcel%7Cvisibility:off&style=feature:administrative.neighborhood%7Cvisibility:off&style=feature:landscape.man_made%7Celement:geometry%7Ccolor:0xffeb3b&style=feature:landscape.natural%7Celement:geometry%7Ccolor:0xdb8836&style=feature:poi.attraction%7Celement:geometry%7Ccolor:0x808080&style=feature:poi.business%7Celement:geometry%7Ccolor:0x808080&style=feature:poi.government%7Celement:geometry.fill%7Ccolor:0x808080&style=feature:poi.medical%7Celement:geometry%7Ccolor:0xff0000&style=feature:poi.park%7Celement:geometry%7Ccolor:0x00ff00&style=feature:poi.place_of_worship%7Celement:geometry%7Ccolor:0x808080&style=feature:poi.school%7Celement:geometry%7Ccolor:0xffffff&style=feature:poi.sports_complex%7Celement:geometry%7Ccolor:0x808080&style=feature:road.arterial%7Celement:geometry%7Ccolor:0x4636db&style=feature:road.highway%7Celement:geometry%7Ccolor:0xdb3636&style=feature:road.local%7Celement:geometry%7Ccolor:0xf44b89&style=feature:water%7Celement:geometry%7Ccolor:0x21d0e0&size=480x360"

POI={
    'human_made':  (255,235,59),
    'natural_land':(219,136,54),
    'high_way':    (219,54,54),
    'two_way':     (70,54,219),
    'one_way':     (244,75,137),
    'water':       (33,208,224),
    'park':        (0,255,0),
    'school':      (255,255,255),
    'medical':     (255,0,0),
    'other_poi':   (128,128,128)
}

# Input parameters

In [3]:
from collections import namedtuple
geoCordinates=namedtuple('geoCordinates',['lat','long'])

In [4]:
#Inputs

# city="Lucknow" #Done
# large_top_left=geoCordinates(26.91850819935238, 80.86420430089606)
# large_bottom_right=geoCordinates(26.789507009635624, 81.0224540389525)
# row,col=10,8

city="Chennai"
large_top_left=geoCordinates(13.16980476617577, 80.16828942014216)
large_bottom_right=geoCordinates(12.948235795868706, 80.29402677340718)
row,col=18,6

#city="Bangalore"
#large_top_left=geoCordinates(13.08341188220246, 77.49618066916169)
#large_bottom_right=geoCordinates(12.89167378353398, 77.68845557490722)
#row,col=15,9

#city="Hyderabad"
#large_top_left=geoCordinates(17.502316259067815, 78.37071274721808)
#large_bottom_right=geoCordinates(17.345203426235784, 78.57123702692401)
#row,col=12,10

#city="Delhi"
#large_top_left=geoCordinates(28.73489222125575, 77.02789452009662)
#large_bottom_right=geoCordinates(28.52733531181014, 77.354735872537)
#row,col=16,16

# Helper classes

In [5]:
class Cell:
    def __init__(self,top_left_cord,bottom_right_cord):
        self.top_left_cord=top_left_cord
        self.bottom_right_cord=bottom_right_cord
        
    def get_Image(self,r,c):
        try:
            if os.path.exists(f'./logs/{city}/Images/{city}_{r}_{c}.png'):
                image=np.array(Image.open(f'./logs/{city}/Images/{city}_{r}_{c}.png'))
            else:
                image=Get_Style_Map_Image_Between(self.top_left_cord,self.bottom_right_cord,style)
                Image.fromarray(image).save(f'./logs/{city}/Images/{city}_{r}_{c}.png') #save image
                
            self.poi_feat=get_poi_feat(image,POI)
            return False
        except Exception as e:
            print(e)
            print("Failed to fetch image ... retrying...")
            sleep(5) #if failed wait for 5 sec
            return True

        
    def __repr__(self):
        return "<"+str(self.top_left_cord)+", "+str(self.bottom_right_cord)+">"

# Creating Axises and Forming Grid Points

In [6]:
x_axis=np.linspace(large_top_left.long,large_bottom_right.long,col+1) #axis in direction to "longitude"
rev_y_axis=np.linspace(large_top_left.lat,large_bottom_right.lat,row+1) #axis in direction to "reverse latitude" for order

X,Y=np.meshgrid(x_axis,rev_y_axis)

# Crawlling

In [7]:
grid=[]
i=0;j=0

#Resume

In [8]:
if i==0:
    pbar = tqdm(total=row*col)

while i<row:
    grid_row=[]
    while j<col:
        cell=Cell((Y[i,j],X[i,j]),(Y[i+1,j+1],X[i+1,j+1])) #corner points lat,long hence Y,X
        
        loop=True
        while loop:
            loop=cell.get_Image(i,j) #if any error occur just loop back here
            
        grid_row.append(cell)
        j+=1;pbar.update(1)
        
    grid.append(grid_row)
    i+=1;j=0;
pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [14:07<00:00,  7.85s/it]


# Printing collected Features

In [9]:
data=[]
for i in range(row):
    for j in range(col):
        d=grid[i][j].poi_feat.copy()
        total=sum(d.values()) #total value
        for k in d.keys():
            d[k]/=total #normalising
        d['row']=i
        d['col']=j
        data.append(d)

df=pd.DataFrame(data)

In [10]:
df[POI.keys()].to_csv(f"./logs/{city}/{city}_spatial_data.csv",index=False)

In [11]:
df[POI.keys()].mean()

human_made      0.683829
natural_land    0.024202
high_way        0.009266
two_way         0.019087
one_way         0.109562
water           0.122317
park            0.017319
school          0.007289
medical         0.000834
other_poi       0.006295
dtype: float64

In [11]:
#NICE